- ref. 

In [1]:
# load data
import pandas as pd, numpy as np
pd.set_option("display.width", 80)
pd.set_option("display.max_rows", 20)
import seaborn as sns, matplotlib.pyplot as plt
import multiprocessing as mp
n_cpus = mp.cpu_count()

# train_df = pd.read_csv("./train.csv")   # 42000
train_df = pd.read_csv("./train.csv", nrows=1000)
test_df = pd.read_csv("./test.csv")   # 28000
train_num = train_df.shape[0]
test_num = test_df.shape[0]
train_x = train_df.drop(['label'], axis=1)
train_y = train_df["label"]
test_x = test_df
all_df = pd.concat((train_x, test_df))

dfd = {'train': train_df, 'test': test_df, 'all': all_df}
for name,df in dfd.items():
    print(name, df.shape)
    print(df.head())

train (1000, 785)
   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0 

In [2]:
# check data
for name,df in dfd.items():
    print(name)
    print(df.info())
    #print(df.describe())   # doesn't show a lot of information

train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 6.0 MB
None
test
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB
None
all
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 173.7 MB
None


In [3]:
# check null
for name,df in dfd.items():
    print(name)
    isnull_ratio = df.isnull().sum()/df.isnull().count()
    print(isnull_ratio[isnull_ratio != 0].sort_values(ascending=False))

Series([], dtype: float64)


Series([], dtype: float64)
all


train
Series([], dtype: float64)
test


In [4]:
# check correlation (takes time)
# -> pixel436 has high correlation with label.. so what?
# correlations = train_df.astype(float).corr()['label'].sort_values(ascending=False)
# print(correlations.head())
# print(correlations.tail())

In [5]:
# check skewness (takes time)
# -> pixel367 has high skewness, but log/boxcox transform doesn't help
# from scipy.stats import skew
# skewness = all_df.apply(lambda x: skew(x)).sort_values(ascending=False)
# print(skewness.head())
# print(skewness.tail())
# all_df['pixel367'].hist(bins=50)
# plt.show()

In [6]:
# grid search (takes time)
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from mlxtend.classifier import StackingClassifier
from sklearn.base import clone

train_x = all_df[:train_num]
test_x = all_df[train_num:]

# models = []
# C_list, gamma_list, degree_list = np.logspace(-2, 0, 3), np.logspace(-3, -1, 3), np.linspace(1, 3, 3)
# grid = [
#     {'C': C_list, 'kernel': ['poly'], 'gamma': gamma_list, 'degree': degree_list},
#     # {'C': C_list, 'kernel': ['sigmoid'], 'gamma': gamma_list},
#     # {'C': C_list, 'kernel': ['linear']},
#     # {'C': C_list, 'kernel': ['rbf'], 'gamma': gamma_list},
#     ]
# models.append(('svm', GridSearchCV(SVC(), grid, cv=3, n_jobs=n_cpus)))
# grid = [{'n_estimators': [1000], 'max_depth': [6,7,8], 'min_samples_leaf': [2,3], 'max_features' : ['sqrt']}]
# models.append(('random_forest', GridSearchCV(RandomForestClassifier(), grid, cv=3, n_jobs=n_cpus)))
# models.append(('extra_trees', GridSearchCV(ExtraTreesClassifier(), grid, cv=3, n_jobs=n_cpus)))
# grid = {'n_estimators': [1000], 'learning_rate' : [0.25, 0.5, 0.75]}
# models.append(('ada_boost', GridSearchCV(AdaBoostClassifier(), grid, cv=3, n_jobs=n_cpus)))
# grid = {'n_estimators': [1000], 'max_depth': [6,7,8], 'min_samples_leaf': [2,3]}
# models.append(('gradient_boosting', GridSearchCV(GradientBoostingClassifier(), grid, cv=3, n_jobs=n_cpus)))
# grid = {'max_iter': [1000], 'tol': [0.001]}
# models.append(('perceptron', GridSearchCV(Perceptron(max_iter=1000, tol=0.001), grid, cv=3, n_jobs=n_cpus)))
# models.append(('sgd_classifier', GridSearchCV(SGDClassifier(max_iter=1000, tol=0.001), grid, cv=3, n_jobs=n_cpus)))
# grid = {'n_neighbors': [2,3,4]}
# models.append(('k-nearest_neighbors', GridSearchCV(KNeighborsClassifier(), grid, cv=3, n_jobs=n_cpus)))
# grid = {}
# models.append(('logistic_regression', GridSerchCV(LogisticRegression(), grid, cv=3, n_jobs=n_cpus)))
# models.append(('gaussian_naive bayes', GridSearchCV(GaussianNB(), grid, cv=3, n_jobs=n_cpus)))
# models.append(('linear_svc', GridSearchCV(LinearSVC(), grid, cv=3, n_jobs=n_cpus)))
# models.append(('decision_tree', GridSearchCV(DecisionTreeClassifier(), grid, cv=3, n_jobs=n_cpus)))

# params = {}
# for name,model in models:
#     model.fit(train_x, train_y)
#     print(name, model.best_score_, model.best_params_)
#     params[name] = model.best_params_
#     # results = model.cv_results_
#     # for mean, std, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
#     #     print("{:0.3f} (+/-{:0.03f} for {}".format(mean, std, params))

# from the result of nrows=20000
params = {
    'k-nearest_neighbors': {'n_neighbors': 3},
    'sgd_classifier': {'max_iter': 1000, 'tol': 0.001},
    'perceptron': {'max_iter': 1000, 'tol': 0.001},
    'gradient_boosting': {'max_depth': 6, 'min_samples_leaf': 3, 'n_estimators': 1000},
    'ada_boost': {'learning_rate': 0.25, 'n_estimators': 1000},
    'extra_trees': {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 1000},
    'random_forest': {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 1000},
    'svm': {'C': 0.01, 'degree': 2.0, 'gamma': 0.001, 'kernel': 'poly'},
    'logistic_regression': {},
    'gaussian_naive bayes': {},
    'linear_svc': {},
    'decision_tree': {},
    }

svm_model = SVC(**params['svm'])
bagging_svm_model = BaggingClassifier(base_estimator=clone(svm_model),
                                      n_estimators=10, max_samples=0.9, max_features=0.2)
et_model = ExtraTreesClassifier(**params['extra_trees'])
bagging_et_model = BaggingClassifier(base_estimator=clone(et_model),
                                     n_estimators=10, max_samples=0.9, max_features=0.2)
knn_model = KNeighborsClassifier(**params['k-nearest_neighbors'])
bagging_knn_model = BaggingClassifier(base_estimator=clone(knn_model),
                                      n_estimators=10, max_samples=0.9, max_features=0.2)
lr_model = LogisticRegression(**params['logistic_regression'])
stacking_model = StackingClassifier(classifiers=[clone(svm_model), clone(et_model), clone(knn_model)],
                                    meta_classifier=clone(lr_model))
models = [
    ('svm', svm_model),
    # ('bagging_svm', bagging_svm_model),
    ('extra_trees', et_model),
    # ('bagging_extra_trees', bagging_et_model),
    ('random_forest', RandomForestClassifier(**params['random_forest'])),
    ('ada_boost', AdaBoostClassifier(**params['ada_boost'])),
    ('gradient_boosting', GradientBoostingClassifier(**params['gradient_boosting'])),
    ('perceptron', Perceptron(**params['perceptron'])),
    ('sgd_classifier', SGDClassifier(**params['sgd_classifier'])),
    ('k-nearest_neighbors', knn_model),
    # ('bagging_k-nearest_neighbors', bagging_knn_model),
    ('logistic_regression', lr_model),
    ('gaussian_naive bayes', GaussianNB(**params['gaussian_naive bayes'])),
    ('linear_svc', LinearSVC(**params['linear_svc'])),
    ('decision_tree', DecisionTreeClassifier(**params['decision_tree'])),
    # ('stacking', stacking_model),
    ]

In [7]:
# cross validation
from sklearn.model_selection import KFold, cross_validate
from multiprocessing import Manager
from sklearn.externals.joblib import Parallel, delayed

k_fold = KFold(n_splits=3)
with Manager() as manager:
    results = manager.list()
    def validate_one_model(name, model):
         # Multiprocessing-backed parallel loops cannot be nested
         scores = cross_validate(model, train_x, train_y, cv=k_fold, scoring='accuracy')
         result = {}
         result['model'] = name
         result['test_mean'] = np.mean(scores['test_score'])
         result['test_std'] = np.std(scores['test_score'])
         result['train_mean'] = np.mean(scores['train_score'])
         result['train_std'] = np.std(scores['train_score'])
         results.append(result)
    Parallel(n_jobs=n_cpus)(delayed(validate_one_model)(name, model) for name,model in models)
    results_merged = {'model': [], 'test_mean': [], 'test_std': [], 'train_mean': [], 'train_std': []}
    for result in results:
        for k,v in result.items():
            results_merged[k].append(v)
    print(pd.DataFrame(results_merged).sort_values(by='test_mean', ascending=False))

                   model  test_mean  test_std  train_mean  train_std
6                    svm   0.877006  0.006369    1.000000   0.000000
9          random_forest   0.862995  0.021172    0.999000   0.000707
8            extra_trees   0.858991  0.010800    0.996000   0.001412
7    k-nearest_neighbors   0.837991  0.010818    0.921499   0.002588
3    logistic_regression   0.813032  0.022873    1.000000   0.000000
0             perceptron   0.813008  0.007285    0.959499   0.005615
4             linear_svc   0.806019  0.020661    1.000000   0.000000
1         sgd_classifier   0.799024  0.018220    0.966494   0.008715
11     gradient_boosting   0.794993  0.010052    1.000000   0.000000
10             ada_boost   0.674033  0.034002    0.828495   0.048280
2   gaussian_naive bayes   0.596971  0.020931    0.737982   0.026358
5          decision_tree   0.587995  0.017499    1.000000   0.000000


In [8]:
# learn, predict and report
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals.joblib import Parallel, delayed, dump
test_id = np.arange(1, test_num+1)
results = {}
def process_one_model(name, model):
    model.fit(train_x, train_y)
    predict_y = model.predict(train_x)
    output = model.predict(test_x)
    results[name] = output
    submit = pd.DataFrame(data={'ImageId':test_id, 'Label':output})
    submit.to_csv('{}_submit.csv'.format(name), index=False)
    #dump(model, '{}.learn'.format(name))
    print(name)
    print(confusion_matrix(train_y, predict_y))
    print(classification_report(train_y, predict_y))
Parallel(n_jobs=n_cpus)(delayed(process_one_model)(name, model) for name,model in models)

[None, None, None, None, None, None, None, None, None, None, None, None]

             precision    recall  f1-score   support

          0       0.92      0.98      0.95       107
          1       0.86      0.99      0.92        96
          2       0.97      0.94      0.96       124
          3       0.89      0.97      0.93        90
          4       0.97      0.90      0.93       102
          5       0.98      0.93      0.95        89
          6       0.97      0.95      0.96        97
          7       0.90      0.91      0.91       105
          8       0.97      0.83      0.90        93
          9       0.87      0.87      0.87        97

avg / total       0.93      0.93      0.93      1000


[[105   0   0   0   0   0   2   0   0   0]
 [  0  95   0   0   0   0   0   0   1   0]
 [  0   3 117   1   0   0   0   3   0   0]
 [  0   0   1  87   0   0   0   1   1   0]
 [  0   1   0   0  92   0   0   2   0   7]
 [  1   1   0   3   0  83   1   0   0   0]
 [  4   1   0   0   0   0  92   0   0   0]
 [  0   4   0   0   2   0   0  96   0   3]
 [  2   3   0   7   0   1   0   0  77   3]
 [  2   2   2   0   1   1   0   5   0  84]]

k-nearest_neighbors


             precision    recall  f1-score   support

          0       0.99      0.91      0.95       107
          1       0.80      0.69      0.74        96
          2       0.90      0.81      0.86       124
          3       0.85      0.74      0.79        90
          4       0.75      0.90      0.82       102
          5       0.58      0.75      0.66        89
          6       0.92      0.71      0.80        97
          7       0.61      0.79      0.69       105
          8       0.63      0.74      0.68        93
          9       0.68      0.48      0.57        97

avg / total       0.78      0.76      0.76      1000


...

[[107   0   0   0   0   0   0   0   0   0]
 [  0  93   0   0   0   0   0   0   2   1]
 [  0   0 120   0   3   0   0   1   0   0]
 [  0   1   1  87   0   0   0   1   0   0]
 [  0   0   0   0 102   0   0   0   0   0]
 [  1   0   0   1   3  79   4   0   1   0]
 [  0   0   0   0   1   0  96   0   0   0]
 [  0   1   0   0   1   0   0 103   0   0]
 [  0   4   0   0   4   1   0   1  81   2]
 [  2   0   0   1  17   0   0   2   0  75]]

sgd_classifier


             precision    recall  f1-score   support

          0       0.99      1.00      1.00       107
          1       0.96      0.98      0.97        96
          2       0.91      1.00      0.95       124
          3       0.95      0.98      0.96        90
          4       0.90      0.98      0.94       102
          5       0.93      0.94      0.94        89
          6       0.98      0.93      0.95        97
          7       0.98      0.93      0.96       105
          8       0.97      0.81      0.88        93
          9       0.91      0.89      0.90        97

avg / total       0.95      0.95      0.95      1000


[[107   0   0   0   0   0   0   0   0   0]
 [  0  94   1   0   0   0   0   0   1   0]
 [  0   0 124   0   0   0   0   0   0   0]
 [  0   0   2  88   0   0   0   0   0   0]
 [  0   1   0   0 100   0   0   0   0   1]
 [  0   0   0   0   1  84   2   0   1   1]
 [  1   1   4   0   0   1  90   0   0   0]
 [  0   1   2   0   1   0   0  98   0   3]
 [  0   1   3   4   2   5   0   0  75   3]
 [  0   0   1   1   7   0   0   2   0  86]]

perceptron


In [9]:
# neural network with tensorflow (skflow -> contrib.learn -> estimator)
import tensorflow as tf
feature_columns = [tf.feature_column.numeric_column(k) for k in train_x.columns]
tf_model = tf.estimator.DNNClassifier(hidden_units=[1000], n_classes=10, feature_columns=feature_columns)
input_fn_train = tf.estimator.inputs.pandas_input_fn(x=train_x, y=train_y, shuffle=False)
tf_model.train(input_fn=input_fn_train, steps=100)
tf_model.evaluate(input_fn=input_fn_train, steps=10)
#scores = cross_validate(tf_model, train_x, train_y, cv=k_fold, scoring='accuracy')   # throws an error
input_fn_predict = tf.estimator.inputs.pandas_input_fn(x=test_x, shuffle=False)
generator = tf_model.predict(input_fn=input_fn_predict)
output = [d['class_ids'][0] for d in generator]
submit = pd.DataFrame(data={'ImageId':test_id, 'Label':output})
submit.to_csv('nn_tensor_submit.csv'.format(name), index=False)

INFO:tensorflow:Restoring parameters from /tmp/tmplm6yoj04/model.ckpt-8


INFO:tensorflow:Saving dict for global step 8: accuracy = 0.603, average_loss = 9.56503, global_step = 8, loss = 1195.63


INFO:tensorflow:Finished evaluation at 2017-11-02-15:09:14


INFO:tensorflow:Evaluation [8/10]


INFO:tensorflow:Evaluation [7/10]


...

INFO:tensorflow:loss = 20356.3, step = 1


INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmplm6yoj04/model.ckpt.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplm6yoj04', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


INFO:tensorflow:Using default config.


In [14]:
# neural network with keras
TODO reshape input
from keras.models import Sequential
from keras.layers.core import Dense, Activation
keras_model = Sequential([
    Dense(input_dim=28*28, units=1000),
    Activation('relu'),
    Dense(units=10),
    Activation('softmax'),
])
keras_model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
keras_model.fit(train_x.values, train_y.values, epochs=10, batch_size=100)
loss, metrics = keras_model.evaluate(train_x.values, train_y.values)
print(metrics)
output = keras_model.predict_classes(test_x.values)
print('\n', output[:10])
submit = pd.DataFrame(data={'ImageId':test_id, 'Label':output})
submit.to_csv('nn_keras_submit.csv'.format(name), index=False)

[7 7 7 7 7 7 7 7 7 7]


27552/28000 [============================>.] - ETA: 0s

26560/28000 [===========================>..] - ETA: 0s

25568/28000 [==========================>...] - ETA: 0s

24640/28000 [=========================>....] - ETA: 0s

...

 800/1000 [=======================>......] - ETA: 0s - loss: 14.3450 - acc: 0.1100

 600/1000 [=================>............] - ETA: 0s - loss: 14.2106 - acc: 0.1183

 420/1000 [===========>..................] - ETA: 0s - loss: 14.1990 - acc: 0.1190

 220/1000 [=====>........................] - ETA: 0s - loss: 14.3593 - acc: 0.1091

Epoch 1/10
  20/1000 [..............................] - ETA: 6s - loss: 15.3122 - acc: 0.0500

In [11]:
!head svm_submit.csv

ImageId,Label
1,2
2,0
3,9
4,9
5,2
6,7
7,0
8,3
9,0
